In [2]:
import time
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from concurrent.futures import ThreadPoolExecutor
import pathlib
import os
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import gc
import threading

### 期間を指定して検索

In [3]:
def search(driver, wait, year: int, month: int):
    URL = "https://db.netkeiba.com/?pid=race_search_detail"
    driver.get(URL)
    time.sleep(1)
    wait.until(EC.presence_of_all_elements_located)
    
    start_year_element = driver.find_element(by=By.NAME, value='start_year')
    start_year_select = Select(start_year_element)
    start_year_select.select_by_value(str(year))
    start_mon_element = driver.find_element(by=By.NAME, value='start_mon')
    start_mon_select = Select(start_mon_element)
    start_mon_select.select_by_value(str(month))
    end_year_element = driver.find_element(by=By.NAME, value='end_year')
    end_year_select = Select(end_year_element)
    end_year_select.select_by_value(str(year))
    end_mon_element = driver.find_element(by=By.NAME, value='end_mon')
    end_mon_select = Select(end_mon_element)
    end_mon_select.select_by_value(str(month))

    # 中央競馬場をチェック
    for i in range(1,11):
        terms = driver.find_element(by=By.ID, value="check_Jyo_"+ str(i).zfill(2))
        terms.click()

    # 表示件数を選択(20,50,100の中から最大の100へ)
    list_element = driver.find_element(by=By.NAME, value='list')
    list_select = Select(list_element)
    list_select.select_by_value("100")

    # フォームを送信
    frm = driver.find_element(by=By.CSS_SELECTOR, value="#db_search_detail_form > form")
    frm.submit()
    time.sleep(5)
    wait.until(EC.presence_of_all_elements_located)

### ページ遷移しながらレースのURL取得

In [4]:
def get_url_with_pagenation(driver, wait, year: int, month: int):

    urls = []
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        all_rows = driver.find_element(by=By.CLASS_NAME, value='race_table_01').find_elements(by=By.TAG_NAME, value="tr")
        for row in range(1, len(all_rows)):
            race_href=all_rows[row].find_elements(by=By.TAG_NAME, value="td")[4].find_element(by=By.TAG_NAME, value="a").get_attribute("href")
            # f.write(race_href+"\n")
            urls.append(race_href)
        try:
            target = driver.find_elements(by=By.LINK_TEXT, value="次")[0]
            driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
        except IndexError:
            break
    return urls

### ThreadPoolで処理

In [14]:
MAX_WORKERS = 10
URL_LIST_PATH = pathlib.Path("url_list")

def job(year: int, month: int):
    
    output_file = URL_LIST_PATH.joinpath(f"{year:d}-{month:02d}.txt")
    
    if output_file.exists():
        return
    
    options = Options()
    options.add_argument('--headless')    # ヘッドレスモードに
    driver = webdriver.Chrome(options=options) 
    wait = WebDriverWait(driver,10)

    search(driver, wait, year, month)
    urls = get_url_with_pagenation(driver, wait, year, month)

    with open(output_file, mode='w') as f:
        f.write("\n".join(urls))
    
    driver.close()
    return

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    for year in range(2008, 2020):
        for month in range(1, 13):
            executor.submit(job, year, month)
    

### レース情報取得

In [20]:
MAX_WORKERS = 30
URL_LIST_PATH = pathlib.Path("url_list")
HTML_PATH = pathlib.Path("html")

def job2(urls_file):

    tok = urls_file.stem.split("-")
    year = int(tok[0])
    month = int(tok[1])
    
    save_dir = HTML_PATH.joinpath(f"{year:d}", f"{month:02d}")
    
    with open(urls_file, "r") as f:
        urls = f.read().splitlines()
        for url in urls:
            list = url.split("/")
            race_id = list[-2]
            response = requests.get(url)
            response.encoding = response.apparent_encoding
            html = response.text
            time.sleep(1)
            
            save_file_path = save_dir.joinpath(f"{race_id}.html")
            save_file_path.parent.mkdir(parents=True, exist_ok=True)
            with open(save_file_path, 'w', encoding = 'utf-8') as file:
                file.write(html)
    return
                
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = [ executor.submit(job2, f) for f in sorted(URL_LIST_PATH.glob("*.txt")) ]

In [23]:
for future in futures:
    future.result()

### htmlをcsvに変換

- メモリリークを起こしているので、threadingで1年分ずつ実行
- それでもピークで10GBくらいは使用するので注意する。

In [2]:
race_data_columns=[
    'race_id',
    'race_round',
    'race_title',
    'race_course',
    'weather',
    'ground_status',
    'time',
    'date',
    'where_racecourse',
    'total_horse_number',
    'frame_number_first',
    'horse_number_first',
    'frame_number_second',
    'horse_number_second',
    'frame_number_third',
    'horse_number_third',
    'tansyo',
    'hukusyo_first',
    'hukusyo_second',
    'hukusyo_third',
    'wakuren',
    'umaren',
    'wide_1_2',
    'wide_1_3',
    'wide_2_3',
    'umatan',
    'renhuku3',
    'rentan3'
    ]

horse_data_columns=[
    'race_id',
    'rank',
    'frame_number',
    'horse_number',
    'horse_id',
    'sex_and_age',
    'burden_weight',
    'rider_id',
    'goal_time',
    'goal_time_dif',
    'time_value',
    'half_way_rank',
    'last_time',
    'odds',
    'popular',
    'horse_weight',
    'tame_time',
    'tamer_id',
    'owner_id'
]

def get_race_and_horse_data_by_html(race_id, html):
    race_list = [race_id]
    horse_list_list = []
    soup = BeautifulSoup(html, 'html.parser')

    # race基本情報
    data_intro = soup.find("div", class_="data_intro")
    race_list.append(data_intro.find("dt").get_text().strip("\n")) # race_round
    race_list.append(data_intro.find("h1").get_text().strip("\n")) # race_title
    race_details1 = data_intro.find("p").get_text().strip("\n").split("\xa0/\xa0")
    race_list.append(race_details1[0]) # race_course
    race_list.append(race_details1[1]) # weather
    race_list.append(race_details1[2]) # ground_status
    race_list.append(race_details1[3]) # time
    race_details2 = data_intro.find("p", class_="smalltxt").get_text().strip("\n").split(" ")
    race_list.append(race_details2[0]) # date
    race_list.append(race_details2[1]) # where_racecourse

    result_rows = soup.find("table", class_="race_table_01 nk_tb_common").findAll('tr') # レース結果
    # 上位3着の情報
    race_list.append(len(result_rows)-1) # total_horse_number
    for i in range(1,4):
        row = result_rows[i].findAll('td')
        race_list.append(row[1].get_text()) # frame_number_first or second or third
        race_list.append(row[2].get_text()) # horse_number_first or second or third

    # 払い戻し(単勝・複勝・三連複・3連単)
    pay_back_tables = soup.findAll("table", class_="pay_table_01")

    pay_back1 = pay_back_tables[0].findAll('tr') # 払い戻し1(単勝・複勝)
    race_list.append(pay_back1[0].find("td", class_="txt_r").get_text()) #tansyo
    hukuren = pay_back1[1].find("td", class_="txt_r")
    tmp = []
    for string in hukuren.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # 枠連
    try:
        race_list.append(pay_back1[2].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")

    # 馬連
    try:
        race_list.append(pay_back1[3].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")

    pay_back2 = pay_back_tables[1].findAll('tr') # 払い戻し2(三連複・3連単)

    # wide 1&2
    wide = pay_back2[0].find("td", class_="txt_r")
    tmp = []
    for string in wide.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # umatan
    race_list.append(pay_back2[1].find("td", class_="txt_r").get_text()) #umatan

    race_list.append(pay_back2[2].find("td", class_="txt_r").get_text()) #renhuku3
    try:
        race_list.append(pay_back2[3].find("td", class_="txt_r").get_text()) #rentan3
    except IndexError:
        race_list.append("0")

    # horse data
    for rank in range(1, len(result_rows)):
        horse_list = [race_id]
        result_row = result_rows[rank].findAll("td")
        # rank
        horse_list.append(result_row[0].get_text())
        # frame_number
        horse_list.append(result_row[1].get_text())
        # horse_number
        horse_list.append(result_row[2].get_text())
        # horse_id
        horse_list.append(result_row[3].find('a').get('href').split("/")[-2])
        # sex_and_age
        horse_list.append(result_row[4].get_text())
        # burden_weight
        horse_list.append(result_row[5].get_text())
        # rider_id
        horse_list.append(result_row[6].find('a').get('href').split("/")[-2])
        # goal_time
        horse_list.append(result_row[7].get_text())
        # goal_time_dif
        horse_list.append(result_row[8].get_text())
        # time_value(premium)
        horse_list.append(result_row[9].get_text())
        # half_way_rank
        horse_list.append(result_row[10].get_text())
        # last_time(上り)
        horse_list.append(result_row[11].get_text())
        # odds
        horse_list.append(result_row[12].get_text())
        # popular
        horse_list.append(result_row[13].get_text())
        # horse_weight
        horse_list.append(result_row[14].get_text())
        # tame_time(premium)
        horse_list.append(result_row[15].get_text())
        # 16:コメント、17:備考
        # tamer_id
        horse_list.append(result_row[18].find('a').get('href').split("/")[-2])
        # owner_id
        horse_list.append(result_row[19].find('a').get('href').split("/")[-2])

        horse_list_list.append(horse_list)

    return race_list, horse_list_list

In [3]:
def job(year: int):

    HTML_PATH = pathlib.Path("html")
    CSV_PATH = pathlib.Path("csv")
    CSV_PATH.mkdir(parents=True, exist_ok=True)

    race_list_all = []
    horse_list_all = []

    for html_file in tqdm(sorted(HTML_PATH.glob(f"{year}/**/*.html")), desc="html", ascii=True):
        race_id = html_file.stem
        with open(html_file, "r", encoding = 'utf-8') as f:
            html = f.read()

            race_list, horses_list = get_race_and_horse_data_by_html(race_id, html)

            race_list_all.append(race_list)
            horse_list_all.extend(horses_list)

    horse_df = pd.DataFrame(horse_list_all, columns=horse_data_columns)
    race_df = pd.DataFrame(race_list_all, columns=race_data_columns)
    horse_df.to_csv(CSV_PATH.joinpath(f"horse_{year}.csv"), index=False)
    race_df.to_csv(CSV_PATH.joinpath(f"race_{year}.csv"), index=False)

In [4]:
for year in range(2008, 2020):
    t = threading.Thread(target=job, args=(year,))
    t.start()
    t.join()

html:   0%|          | 0/3451 [00:00<?, ?it/s]

html:   0%|          | 0/3454 [00:00<?, ?it/s]

html:   0%|          | 0/3454 [00:00<?, ?it/s]

html:   0%|          | 0/3455 [00:00<?, ?it/s]

html:   0%|          | 0/3454 [00:00<?, ?it/s]

html:   0%|          | 0/3452 [00:00<?, ?it/s]

### 各年のcsvを結合

In [3]:
CSV_PATH = pathlib.Path("csv")
horse_df = pd.concat(
    [pd.read_csv(csv_file, low_memory=False) for csv_file in CSV_PATH.glob("horse_*")]
, ignore_index=True)
race_df = pd.concat(
    [pd.read_csv(csv_file, low_memory=False) for csv_file in CSV_PATH.glob("race_*")]
, ignore_index=True)

In [4]:
horse_df.to_csv(CSV_PATH.joinpath("horse.csv"), index=False)
race_df.to_csv(CSV_PATH.joinpath("race.csv"), index=False)

In [5]:
race_df

,race_id,race_round,race_title,race_course,weather,ground_status,time,date,where_racecourse,total_horse_number,...,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3
0,200806010101,1 R,3歳未勝利,ダ右1200m,天候 : 曇,ダート : 良,発走 : 09:50,2008年1月5日,1回中山1日目,16,...,400,290,"3,480","4,520","1,550",960,"1,560","8,940","10,600",0
1,200806010102,2 R,3歳未勝利,ダ右1800m,天候 : 曇,ダート : 良,発走 : 10:20,2008年1月5日,1回中山1日目,16,...,"1,620",460,"11,680","12,460","3,160","1,230","13,320","16,480","68,590",0
2,200806010103,3 R,3歳未勝利,ダ右1200m,天候 : 曇,ダート : 良,発走 : 10:50,2008年1月5日,1回中山1日目,16,...,250,470,390,"1,010",430,960,"2,650","1,430","5,320",0
3,200806010104,4 R,3歳新馬,ダ右1200m,天候 : 晴,ダート : 良,発走 : 11:20,2008年1月5日,1回中山1日目,16,...,230,460,"2,000","2,870",990,"1,660","1,360","6,380","13,300",0
4,200806010105,5 R,3歳未勝利,芝右 外1600m,天候 : 晴,芝 : 良,発走 : 12:10,2008年1月5日,1回中山1日目,16,...,160,170,420,460,230,240,530,720,870,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41435,201909050908,8 R,3歳以上1勝クラス,芝右 外1600m,天候 : 晴,芝 : 良,発走 : 14:01,2019年12月28日,5回阪神9日目,18,...,160,970,"2,800","2,180",800,"5,500","2,610","5,450","23,640","135,810"
41436,201909050909,9 R,春待月賞,ダ右1400m,天候 : 晴,ダート : 稍重,発走 : 14:31\n\n過去の春待月賞,2019年12月28日,5回阪神9日目,16,...,"1,830",220,790,"24,880","10,680","1,400","7,940","48,320","75,580","562,740"
41437,201909050910,10 R,フォーチュンカップ,芝右2000m,天候 : 晴,芝 : 良,発走 : 15:05\n\n過去のフォーチュンカップ,2019年12月28日,5回阪神9日目,16,...,160,360,320,550,270,690,"1,340",840,"2,600","7,710"
41438,201909050911,11 R,ベテルギウスS(L),ダ右1800m,天候 : 曇,ダート : 稍重,発走 : 15:45\n\n過去のベテルギウスS,2019年12月28日,5回阪神9日目,16,...,120,390,490,550,250,"1,140",800,"1,380","3,430","16,570"


In [6]:
horse_df

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,time_value,half_way_rank,last_time,odds,popular,horse_weight,tame_time,tamer_id,owner_id
0,200806010101,1,1,2,2005102028,牝3,54.0,733,1:13.9,NaN,\n\n**\n\n,1-1,39.2,8.1,4.0,450(-10),\n\n,138,498800
1,200806010101,2,3,5,2005101044,牝3,54.0,663,1:13.9,ハナ,\n\n**\n\n,10-7,38.5,13.4,6.0,482(+2),\n\n,1088,163002
2,200806010101,3,2,3,2005104156,牝3,54.0,684,1:14.2,1.3/4,\n\n**\n\n,13-8,38.5,8.2,5.0,458(+4),\n\n,1054,486800
3,200806010101,4,4,7,2005103612,牝3,54.0,1095,1:14.3,3/4,\n\n**\n\n,2-2,39.5,2.9,1.0,456(-2),\n\n,436,274002
4,200806010101,5,6,12,2005102035,牝3,54.0,1077,1:14.5,1.1/4,\n\n**\n\n,3-3,39.6,4.3,2.0,458(-10),\n\n,1007,498800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596058,201909050912,12,5,9,2014106179,牡5,57.0,1102,1:10.4,3/4,\n\n**\n\n,9-10,34.4,11.5,5.0,490(0),\n\n,1055,506800
596059,201909050912,13,2,4,2012101393,牝7,55.0,1138,1:10.5,1/2,\n\n**\n\n,8-6,34.7,294.0,15.0,458(-6),\n\n,1006,962033
596060,201909050912,14,5,10,2015100672,牡4,57.0,1037,1:10.7,1.1/4,\n\n**\n\n,1-1,35.8,114.8,13.0,482(-6),\n\n,1078,788800
596061,201909050912,15,7,14,2014103559,牝5,55.0,1180,1:11.0,1.3/4,\n\n**\n\n,15-15,34.2,115.2,14.0,438(-4),\n\n,1095,36006
